In [1]:
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GenerationConfig

/fsx/users/tcbegley/conda/envs/rl-stable/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [3]:
model = GPT2LMHeadModel.from_pretrained("out_rlhf").eval().to("cuda")

In [4]:
dataset = load_dataset("CarperAI/openai_summarize_tldr", split="test")

Found cached dataset parquet (/data/home/tcbegley/.cache/huggingface/datasets/CarperAI___parquet/CarperAI--openai_summarize_tldr-536d9955f5e6f921/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [5]:
generation_config = GenerationConfig(
    max_new_tokens=50,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

In [10]:
example = dataset[42]
tokenized_prompt = tokenizer(example["prompt"], return_tensors="pt").to("cuda")

res = model.generate(
    **tokenized_prompt,
    generation_config=generation_config,
)

string_to_write = (
    "Query: \n"
    f"{example['prompt']}\n"
    f"Generated: \n"
    f"{tokenizer.decode(res[0, len(tokenized_prompt['input_ids'][0]):].cpu().numpy().tolist())}\n"
    f"True label: \n"
    f"{example['label']}\n"
    f"====================================================\n"
)
print(string_to_write)

Query: 
SUBREDDIT: r/relationships
TITLE: My (20M) disinterest in sex is really hurting my girlfriend (19F) of 3 years
POST: My girlfriend and I started as a long distance relationship when we were in high school. We met three times over the years, each time for about a week, before moving in together during the beginning of this year. Initially, our sex life was pretty great. We had sex all the time and everything was fine. Lately, though, I've been wanting to less and less and it's really hurting her self esteem and making her feel like I'm unattracted to her. 

This may get a bit graphic, I just want to let you guys know why I'm not really that into it anymore in the hopes that you'll have some kind of advice as to what I can do. So generally, when we begin having sex, it's awesome. But after about a minute or so, it's like my dick goes numb. It makes it very difficult to stay into it, even though I enjoy pleasuring her. This makes me kind of dread having sex because I know it'll ju